<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd


from pymystem3 import Mystem
import re
import torch
import transformers
import pymorphy2

from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer

from tqdm import notebook
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

from sklearn.metrics import accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
#читаю датасет

data=pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
#проверяю что в нем лежит

data

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


много ненужных симбволов типа троеточий, разделителей \n, таке лишний столбец 'Unnamed: 0'

In [4]:
#удаляю столбец

data = data.drop('Unnamed: 0', axis=1)

In [5]:
#провряю удаление

data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


In [7]:
#функция очистки текста

def clear_text(text):
    text = text.lower()
    cl_text = re.sub(r"[^a-zA-Z' ]", " ", text)
    clear_list = cl_text.split()
    clear_text = ' '.join(clear_list)
    return clear_text
 
#функция лематизации

wnl = WordNetLemmatizer()
def lemmatize(text):
    lemm_list = [wnl.lemmatize(w) for w in nltk.word_tokenize(text)] 
    lemm_text = " ".join(lemm_list)
    return lemm_text
 


In [8]:
#сама лематизация

lemm=[]
for i in tqdm(range(len(data['text']))):
    lemm.append(lemmatize(clear_text(data['text'][i])))
    
data['lemm_text']=pd.Series(lemm, index=data.index)

100%|██████████| 159292/159292 [01:26<00:00, 1838.27it/s]


In [9]:
#проверяю лематизацию

data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww he match this background colour i 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i 'm really not trying to edit war it ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [10]:
#проверяю баланс классов

data['toxic'].mean()

0.10161213369158527

Классы несбалансированы.

вывод 1:
    1. прочитал датасет - 
    2. почистил от регулярных выражений
    3. провел лемматизацию
    4. обнаружил дисбаланс классов - при работе с моделями этот момент исправлю
    

## Обучение

In [11]:
#разбиваю на выборки

features = data['lemm_text']
target=data['toxic']

In [12]:
#делю сначала на test и  train_valid

features_train_valid, features_test, target_train_valid, target_test = train_test_split(features, target, shuffle=True,
                                                                            test_size = 0.1, random_state = 12345)

In [13]:
# потом train_valid  на train и valid

features_train, features_valid, target_train, target_valid = train_test_split(features_train_valid, target_train_valid, 
                                                                              shuffle=True, test_size=0.5, 
                                                                              random_state = 12345)

In [14]:
#проверяю

print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(71681,)
(71681,)
(15930,)


In [15]:
#загружаю список стоп слов

nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
features_train_no_tfidf = features_train.copy()
features_valid_no_tfidf = features_valid.copy()
features_test_no_tfidf = features_test.copy()

In [17]:
#создаю счётчик, указав в нём стоп-слова и чтобы посчитать TF-IDF, вызоваяю функцию fit_transform():

features_train = count_tf_idf.fit_transform(features_train)
features_valid = count_tf_idf.transform(features_valid)
features_test = count_tf_idf.transform(features_test)


In [18]:
%%time
#смотрю модель LogisticRegression

model = LogisticRegression(class_weight='balanced', random_state=12345, C = 4, penalty = 'l1', solver='liblinear', max_iter=200)
model.fit(features_train, target_train)
predict = model.predict(features_valid)
print(f'f1_score {f1_score(target_valid, predict)}')
print()


f1_score 0.7540794038288579

CPU times: user 1.5 s, sys: 47.5 ms, total: 1.55 s
Wall time: 1.55 s


модель LogisticRegression, f1_score = 0.75

In [36]:
%%time

#LogisticRegression c указанными гиперпарамтерами

model = LogisticRegression(class_weight='balanced', 
                           random_state=12345,
                           C = 2,
                           penalty = 'l1',
                           solver='liblinear',
                           max_iter=50)

f1_score_lg = cross_val_score(model,
                           features_train,
                           target_train,
                           cv=3,
                           scoring='f1').mean()                      
print(f'f1_score {f1_score_lg}')
print()

f1_score 0.7499512852373286

CPU times: user 1.66 s, sys: 319 µs, total: 1.66 s
Wall time: 1.66 s


 LogisticRegression, f1_score = 0.755 с кросвалидацией

In [20]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier

clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SGDClassifier(class_weight="balanced"))
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 3)],
    'clf__loss': ['modified_huber', 'hinge'],
}

grid_search = GridSearchCV(estimator=clf, param_grid=parameters,
                           scoring='f1', n_jobs=-1, verbose=1)



CPU times: user 2.88 ms, sys: 0 ns, total: 2.88 ms
Wall time: 1.94 ms


features_train = features_train.lower()

<div class="alert alert-info">
<b>Комментарий студента</b>
не помогло
</div>

In [21]:
%%time

grid_search.fit(features_train_no_tfidf, target_train)



Fitting 5 folds for each of 4 candidates, totalling 20 fits
CPU times: user 5min 12s, sys: 12.4 s, total: 5min 24s
Wall time: 5min 25s


GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('clf',
                                        SGDClassifier(class_weight='balanced'))]),
             n_jobs=-1,
             param_grid={'clf__loss': ['modified_huber', 'hinge'],
                         'vect__ngram_range': [(1, 1), (1, 3)]},
             scoring='f1', verbose=1)

In [22]:
type(features_train)

scipy.sparse._csr.csr_matrix

In [23]:
%%time

grid_search.best_estimator_.fit(features_train_no_tfidf, target_train)



CPU times: user 3.99 s, sys: 171 ms, total: 4.16 s
Wall time: 4.17 s


Pipeline(steps=[('vect', CountVectorizer()),
                ('clf',
                 SGDClassifier(class_weight='balanced',
                               loss='modified_huber'))])

In [28]:
%%time

predict_SGDC = grid_search.best_estimator_.predict(features_valid_no_tfidf)

CPU times: user 2.78 s, sys: 12.4 ms, total: 2.8 s
Wall time: 2.8 s


In [29]:
%%time

print(f'f1_score {f1_score(predict_SGDC,target_valid)}')
print()


f1_score 0.7346013351807533

{'clf__loss': 'modified_huber', 'vect__ngram_range': (1, 1)}

CPU times: user 19 ms, sys: 103 µs, total: 19.1 ms
Wall time: 17.4 ms


GDClassifier с GridSearchCV показал f1=0.73 

итого лучшая модель LogisticRegression, f1_score = 0.75

In [32]:
%%time
#смотрю модель LogisticRegression

model = LogisticRegression(class_weight='balanced', random_state=12345, C = 4, penalty = 'l1', solver='liblinear', max_iter=200)
model.fit(features_train, target_train)
predict_lg = model.predict(features_test)
print(f'f1_score {f1_score(target_test, predict_lg)}')
print()

f1_score 0.7608505680163123

CPU times: user 1.45 s, sys: 20.1 ms, total: 1.47 s
Wall time: 1.47 s


лучшая модель по f1 = 0.76  - LogisticRegression на тестовых данных

## Выводы

вывод:
    1. загружен датасет, проведена предобработка
    2. выбраны модели классификации, для них подготовил выборки, обучил модели, посчитал нужную метрику качества f1 не меньше 0.75. 
    3. лучшей моделью с f1 = 0.94 оказалась SGDClassifier с подбором параметров через GridSearchCV на тестовых данных